
# Practice Lab M03 (Version 3)
## Data Acquisition and Data Source

This task on data acquisition and data source is mainly to focus on how to properly acquire and read the data with differnt formats.

To do:

*   Read the given data for required format; 
*   Using Numpy and Pandas to do the ETL for the data.


**Task 1.1** We first do a simple task on crawling the data from html -- the Deakin IT staff page. To do this, we need to understand the structure of the html as well as the web crawling library -- selenium or beautiful soup. 
In this task, please find all professors (Emeritus Professors, Professors and Associate Professors only) in Schoolf of IT and save it as csv 

**Background:**
Selenium Python bindings provides a simple API to write functional/acceptance tests using Selenium WebDriver. -- [check link](https://selenium-python.readthedocs.io/installation.html#introduction)

**Hint:**

There are several steps to crawl the HTML:


1.   Building a web driver from Selenium and use the get function to call the page.  
        ```
        wd = webdriver.Chrome('chromedriver',options=options)
        ***wd.get("https://www.deakin.edu.au/information-technology/staff-listing")***
        ```
2.   Find the table in the page and then further format the content into csv 
        ```
       shapes = wd.find_elements_by_tag_name('table')
       for table in shapes[:3]:
          for tr in table.find_elements_by_tag_name('tr'):
              for td in tr.find_elements_by_tag_name('td'):
                  for a in td.find_elements_by_tag_name('a'):
        ```


In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)

In [ ]:
# print the content from the tables which contain the professors information
wd.get("https://www.deakin.edu.au/information-technology/staff-listing")
shapes = wd.find_elements_by_tag_name('table')
for table in shapes[:3]:
  for tr in table.find_elements_by_tag_name('tr'):
    for td in tr.find_elements_by_tag_name('td'):
        for a in td.find_elements_by_tag_name('a'):
            print(a.get_attribute('text'))

In [ ]:
# We need to create a parser to seperate the information, for example, we need to extract the title and name for each professor
def parse_name(stringtext):
  return " ".join(stringtext.split(" ")[-2:])," ".join(stringtext.split(" ")[:-2])

In [ ]:
# let's store the name and title into a new dataframe.
import pandas as pd
data = []
for table in shapes[:3]:
  for tr in table.find_elements_by_tag_name('tr'):
    #tds = tr.find_elements_by_tag_name('td')
    for td in tr.find_elements_by_tag_name('td'):
        for a in td.find_elements_by_tag_name('a'):
          data.append(parse_name(a.get_attribute('text')))
df = pd.DataFrame(data)
df['University'] = 'Deakin University'
df.columns = ['Name','Title','University']
df.to_csv('data/Professor_name_list.csv', )

In [ ]:
#Below code is the function to fetch the google scholar profile of each professor
#What it does are: 
#1 searching the google scholar for all professors to obtain their citations_all, h-index_all, i10_all, citation_since2016 h-index-2016 and i10_since2016.
#2 Saving the results as csv and must having all the professors name, title and all 6 citation information.
#3 If the professors did not have the google scholar profle with the name, save the 6 citation information as string "na"
#4 The input of the function below is in a format of ('firstname+lastname+deakin+university') -- which is a preferable format for google scholar to search
from time import sleep
def fetch_google_scholar_profile(input):
  statistics = []
  url = str("https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors="+input+"&btnG=")
  wd.get(url)
  #sleep(5)
  bodyText = wd.find_element_by_tag_name("body").text
  if "didn't match any user profiles" in bodyText:
    statistics.extend(['na', 'na', 'na','na','na','na'])
  else:
    elems = wd.find_element_by_class_name("gs_ai_pho")
    profile_url = elems.get_attribute("href")
    wd.get(profile_url)
    table = wd.find_element_by_id("gsc_rsb_st")
    for td in table.find_elements_by_class_name('gsc_rsb_std'):
      #for td in tr.find_elements_by_tag_name('td'):
      #  a = td.find_element_by_tag_name('a')
      #tds = tr.find_elements_by_class_name('gsc_rsb_std')
      statistics.append(td.get_attribute('textContent'))
  return statistics

In [ ]:
#Now, let's collect the citation information for all professors we have crawled.
citation_info = []
for i in range(len(df.Name)):
  author = df.Name[i].split(" ",1)[0]+"+"+df.Name[i].split(" ",1)[1]+"+deakin+"+"university"
  print(author)
  citation = fetch_google_scholar_profile(author)
  citation.insert(0, df.Title[i])
  citation.insert(0, df.Name[i])
  citation_info.append(citation)
  print('finished') 

In [ ]:
#Let's store the citation information into csv 
import numpy as np
df_citation = pd.DataFrame(np.array(citation_info))
df_citation.columns = ['Name','Title','citation_all','citation_since2016','h-index_all','h-index_since2016','i10-index_all','i10-index_since2016']
df_citation.to_csv('data/Professor_citation_informaton.csv')

**Task 1.2** After acquiring the dataframe for the html, let's first focus on the dictionary format -- which is the common used format while for semi-structure data in frontend development. **A record in dictionary  (one row in csv format) is now partitioned on column level**. 
We will try to first convert the Professor_citation_informaton.csv to 
the in memory dictionary (removing 'na'). Then we will 
format the dictionary by only selecting the **records (rows)** with citation_all > 3000 and citation_all < 10000  and then store it as a json file.

**Background:** 

Dictionaries are used to store data values in key:value pairs.
A dictionary is a collection which is ordered*, changeable and does not allow duplicates. -- [check link](https://www.w3schools.com/python/python_dictionaries.asp)


**Hint:**

```
for key, val in d.items():
    if filter_string not in key:
        continue
    do something

```

or 
```
filtered_dict = {k:v for (k,v) in d.items() if filter_string in k}
```

In [ ]:
#Firstly, we will remove the 'na' from the df_citation and convert it to dictionary.


In [ ]:
# Let's first print out all the keys and the length of the values in the given data dictionary


In [ ]:
# let's print the keys and also the unique value from the values in in each (k,v) pair in the given data dictionary
# unique value of the array could be calculated via numpy.unique(array,return_counts = False)


In [ ]:
# Let's then print both keys and the length of the unique value from the values in each (k,v) pair in the given data dictionary
# for example, in the key value pair {fruit: ['apple','pear','banana']}, the length of the unique value is 3
# unique value of the array could be calculated via numpy.unique(array,return_counts = False)
import numpy as np



In [ ]:
# Now let's filter the dictionary, by doing it, you need to first create a null dictionary 
# and then write the filtered key value pair in the null dictionary
# The code is given as below

newDict = dict()
for k,v in dic.items():
   # Check if value meets the condition on particular key
    if k == 'citation_all':
        flt = filter(lambda citation: citation < 10000 and citation >3000, np.array(v).astype(int))
        new_v = list(flt)
        newDict[k] = new_v
    else:
        newDict[k] = v
      

In [ ]:
# let's double check the results
for k, v in newDict.items():
  print(k, np.unique(v,return_counts=False))

**Question here for 1.2** Have we finished the task 1.2? if yes, why? if no, why?

In [ ]:
# Now let's redo the filtering on the dictionary, by doing it, you need to first create a null dictionary 
# and then write the filtered key value pair in the null dictionary
# Please write code as below again:

newDict = dict()
index_col = [i for i in range(len(dic['citation_all'])) if (np.array(dic['citation_all']).astype('int')[i] > 3000) and (np.array(dic['citation_all']).astype('int')[i] <10000)]
for k,v in dic.items():   
    newDict[k] = list(np.array(v)[index_col])


In [ ]:
#let's check the length of the values again, do we see the difference?
for k, v in newDict.items():
  print(k, len(v))

In [ ]:
# let's store the filtered result into json,
# the numpy to json encoder is provided
import json
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open('data/citation.json', 'w') as fp:
    json.dump(newDict, fp, default=np_encoder)

In [ ]:
#let's read it via pandas to check
df_json = pd.read_json('data/citation.json')
df_json

**Task 1.3** Now we have the the json file, then next step for us is doing the ETL by using popular SQL. 
SQL is a domain-specific language used in programming and designed for managing data held in a relational database management system, or for stream processing in a relational data stream management system.
In this task,
we will review some simple sql query by using pandas dataframe.


In [ ]:
# firstly, let's read the newplayer.json into dataframe by using pandas
import pandas as pd
df_citation = pd.read_json('data/citation.json')
df_citation

In [ ]:
# install pandasql
!pip install -U pandasql

In [ ]:
# let's first run a select query for the dataframe
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

query1 = pysqldf("SELECT * FROM df_citation LIMIT 10;")
query1

In [ ]:
# let's do some transformation on the column 'citation_all' with conditions for df_newplayer
# if 0<citation_all<=4000 then 'low'; 4000<citation_all<=4300 then 'medium'; 4300<citation_all<=7500 then 'good'; 7500<citation_all<=10000 then 'excellent';
query2 = pysqldf("""

""")
query2.head(5)

In [ ]:
# let's calculate the average value of 'citation_all', 'citation_since2016' by group the Title

query3 = pysqldf("""

""")
query3.head(5)


**Task 1.4** Could we do the similar ETL as task 1.3 in pandas?
Such as group by the Name and Title by having the average value of 'citation_all', 'citation_since2016'

**Hint:**

using groupby() from pandas

In [ ]:
# firstly, let's select the columns from df_citation
df_sel = df_citation[['Name','Title','citation_all', 'citation_since2016']]

In [ ]:
# then let's create the aggregation by using pandas


In [ ]:
# let's reset the index to format the aggregated dataframe


In [ ]:
# then let's convert citation_all to categorical data type
# the conditions for citation_all are if 0<citation_all<=4000 then 'low'; 4000<citation_all<=4300 then 'medium'; 4300<citation_all<=7500 then 'good'; 7500<citation_all<=10000 then 'excellent';

conditions1 = [
    (df_sel['citation_all']>0) & (df_sel['citation_all']<=4000),
    (df_sel['citation_all']>4000) & (df_sel['citation_all']<=4300),
    (df_sel['citation_all']>4300) & (df_sel['citation_all']<=7500),
    (df_sel['citation_all']>7500) & (df_sel['citation_all']<=10000)
]

choices = ['low','medium','good','excellent']



**Task 1.5 (Advanced)** Now we have the dataframe on both numerical and categorical datatype. For many big analysis, category datatype is not the best format to start with. The common way to deal category datatype is to transform it to one hot encode format (only with 1 and 0). Could you finish the one hot encode transforming for categorical column by using the provided code in hint?

**Background:**

One-hot Encoding is a type of vector representation in which all of the elements in a vector are 0, except for one, which has 1 as its value, where 1 represents a boolean specifying a category of the element. -- [check link](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

**Hint:**

```
df_onehot = pd.get_dummies(s)
```

In [ ]:
# then let's find and remove the numerical columns from all columns


In [ ]:
# let's use the code in hint to do the one hot encode and print the dataframe


In [ ]:
# let's combine the new onehot encode dataframe with numerical dataframe to obtain the full dataframe df_all


##**Tasks 2 Advanced Data Acquisition **

This task on Advanced data acquisition is to use numpy and pandas to perform more advanced Code-based ETL.

To do:

*   Create a function to calculate the euclidean distance between recoard; 
*   Find the most similar record for each one in the bank data.

**Task 2.1** In numpy, the euclidean distance could be calculated via 
```
np.sqrt(np.sum(np.square(point1 - point2)))
```
point1 and point2 is the 1D array, please folllow the above calculation and build a function to calculate the euclidean distance for any two arrays from a given dataframe.

In [ ]:
# define the funtion as below with name "dist_func"

def dist_func(row1,row2):
  return np.sqrt(np.sum(np.square(row1 - row2)))

In [ ]:
# what about if point2 is a 2d array? how to calculate the distance from point1 to each dimension of point2?



**Task 2.2** Now, we will need to calculate the euclidean distance between each row and all the rows (let's include the current row at here), also we would like to save the distances into array for each row. To the end, you will have a distance matrix with shape of (n,n) where n is the total rows.
We will use *df_all* as the input.

**Hint:**

Use the for loop on each row could be a good start

In [ ]:
# let's write the code here:



In [ ]:
# let's print the distance matrix


In [ ]:
# let's find the index of the smallest distance for each row in the distance


In [ ]:
#let's put the results into pandas dataframe
